In [1]:
library(tidyr)
library(dplyr)
library(ggplot2)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
similarity <- read.csv('../../interneuron-reward-data/processed/ideal_projection_similarity_trial_resolved.csv')

# Convert numeric cell_id to factor
similarity[, 'day'] <- as.factor(similarity[, 'day'])
similarity[, 'cell_id'] <- as.factor(similarity[, 'cell_id'])
similarity[, 'trial_id'] <- as.factor(similarity[, 'trial_id'])

# Display first few rows
head(similarity)

day,cell_type,cell_id,mouse_id,trial_id,tone,tone...delay,delay,reward
1,vip,3121,CL136,5094,0.13798489,-0.027988964,-0.14793946,0.03789182
1,vip,3121,CL136,5095,0.11568627,0.176498293,0.12313947,-0.07155153
1,vip,3121,CL136,5096,-0.06912943,0.001183833,0.05824727,0.10211240
1,vip,3121,CL136,5097,0.09978416,0.022661767,-0.05395247,0.01946407
1,vip,3121,CL136,5098,0.07888381,0.032715124,-0.02435786,-0.01549540
1,vip,3121,CL136,5099,0.02274820,-0.038963612,-0.06684784,-0.08558579


In [3]:
reward_change_by_cell <- similarity %>% 
    group_by(cell_id) %>% 
    summarize(
        cell_id=cell_id[1],
        cell_type=cell_type[1],
        mouse_id=mouse_id[1],
        greater_p_value=ifelse(
            (sum(day == 1) > 0) & (sum(day == 7) > 0),
            wilcox.test(
                reward[day == 1], 
                reward[day == 7],
                alternative='greater',
                paired=FALSE,
                na.action=na.exclude
            )$p.value,
            NA
        ),
        less_p_value=ifelse(
            (sum(day == 1) > 0) & (sum(day == 7) > 0),
            wilcox.test(
                reward[day == 1], 
                reward[day == 7],
                alternative='less',
                paired=FALSE,
                na.action=na.exclude
            )$p.value,
            NA
        )
    )

`summarise()` ungrouping output (override with `.groups` argument)


In [4]:
head(reward_change_by_cell)

cell_id,cell_type,mouse_id,greater_p_value,less_p_value
1,pyr,CL174,0.6676533382,3.367961e-01
2,pyr,CL174,0.2934068242,7.107881e-01
3,pyr,CL174,0.0007741467,9.992600e-01
4,pyr,CL174,0.0262814261,9.744726e-01
5,pyr,CL174,0.9982381444,1.836443e-03
6,pyr,CL174,0.9999981558,1.976601e-06


In [5]:
omnibus_test <- function(num_significant_p_values, num_tests, alpha_level) {
    if (num_significant_p_values == 0) {
        return(1.0)
    } else {
        return(
            pbinom(num_significant_p_values - 1, num_tests, alpha_level, lower.tail=FALSE)
        )
    }
}

In [7]:
omnibus_p_vals <- reward_change_by_cell %>%
    group_by(cell_type, mouse_id) %>%
    summarize(
        cell_type=cell_type[1],
        mouse_id=mouse_id[1],
        num_greater=sum(greater_p_value <= 0.05, na.rm=TRUE),
        num_less=sum(less_p_value <= 0.05, na.rm=TRUE),
        num_cells=length(greater_p_value),
        greater_omnibus_p=omnibus_test(
            sum(greater_p_value <= 0.05, na.rm=TRUE),
            length(greater_p_value),
            0.05
        ),
        less_omnibus_p=omnibus_test(
            sum(less_p_value <= 0.05, na.rm=TRUE),
            length(less_p_value),
            0.05
        )
    )

omnibus_p_vals[, 'pct_greater'] <- omnibus_p_vals[, 'num_greater'] / omnibus_p_vals[, 'num_cells']
omnibus_p_vals[, 'pct_less'] <- omnibus_p_vals[, 'num_less'] / omnibus_p_vals[, 'num_cells']


`summarise()` regrouping output by 'cell_type' (override with `.groups` argument)


In [8]:
omnibus_p_vals

cell_type,mouse_id,num_greater,num_less,num_cells,greater_omnibus_p,less_omnibus_p,pct_greater,pct_less
pv,CL147,39,5,102,1.861922e-24,5.817365e-01,0.38235294,0.04901961
pv,CL158,7,1,17,9.728422e-06,5.818797e-01,0.41176471,0.05882353
pv,CL172,4,13,45,1.866167e-01,1.959052e-07,0.08888889,0.28888889
pv,CL195,0,23,40,1.000000e+00,4.593372e-20,0.00000000,0.57500000
pv,CL196,32,6,61,1.203155e-25,8.380005e-02,0.52459016,0.09836066
pv,CL198,0,23,47,1.000000e+00,5.922695e-18,0.00000000,0.48936170
pyr,CL174,124,15,211,3.959208e-103,1.096242e-01,0.58767773,0.07109005
pyr,CL175,22,6,73,5.053582e-12,1.578928e-01,0.30136986,0.08219178
pyr,CL176,48,91,299,1.258279e-12,3.335544e-45,0.16053512,0.30434783
pyr,CL181,118,3,183,3.462029e-105,9.952414e-01,0.64480874,0.01639344


In [29]:
omnibus_p_vals[, 'greater_significance'] <- omnibus_p_vals[, 'greater_omnibus_p'] < (0.05 / 16)
omnibus_p_vals[, 'less_significance'] <- omnibus_p_vals[, 'less_omnibus_p'] < (0.05 / 16)

In [30]:
omnibus_p_vals

cell_type,mouse_id,greater_omnibus_p,less_omnibus_p,greater_significance,less_significance
pv,CL147,1.861922e-24,5.817365e-01,TRUE,FALSE
pv,CL158,9.728422e-06,5.818797e-01,TRUE,FALSE
pv,CL172,1.866167e-01,1.959052e-07,FALSE,TRUE
pv,CL195,1.000000e+00,4.593372e-20,FALSE,TRUE
pv,CL196,1.203155e-25,8.380005e-02,TRUE,FALSE
pv,CL198,1.000000e+00,5.922695e-18,FALSE,TRUE
pyr,CL174,3.959208e-103,1.096242e-01,TRUE,FALSE
pyr,CL175,5.053582e-12,1.578928e-01,TRUE,FALSE
pyr,CL176,1.258279e-12,3.335544e-45,TRUE,TRUE
pyr,CL181,3.462029e-105,9.952414e-01,TRUE,FALSE


In [34]:
omnibus_p_vals <- reward_change_by_cell %>%
    group_by(cell_type) %>%
    summarize(
        cell_type=cell_type[1],
        num_greater=sum(greater_p_value <= 0.05, na.rm=TRUE),
        num_less=sum(less_p_value <= 0.05, na.rm=TRUE),
        num_cells=length(greater_p_value),
        greater_omnibus_p=omnibus_test(
            sum(greater_p_value <= 0.05, na.rm=TRUE),
            length(greater_p_value),
            0.05
        ),
        less_omnibus_p=omnibus_test(
            sum(less_p_value <= 0.05, na.rm=TRUE),
            length(less_p_value),
            0.05
        )
    )

omnibus_p_vals[, 'pct_greater'] <- omnibus_p_vals[, 'num_greater'] / omnibus_p_vals[, 'num_cells']
omnibus_p_vals[, 'pct_less'] <- omnibus_p_vals[, 'num_less'] / omnibus_p_vals[, 'num_cells']

omnibus_p_vals

`summarise()` ungrouping output (override with `.groups` argument)


cell_type,num_greater,num_less,num_cells,greater_omnibus_p,less_omnibus_p,pct_greater,pct_less
pv,82,71,312,1.037671e-35,5.541947e-27,0.2628205,0.2275641
pyr,377,282,1207,9.434181e-186,9.739341e-105,0.3123447,0.2336371
vip,64,170,407,7.436484e-16,1.969450e-108,0.1572482,0.4176904


In [35]:
tone_change_by_cell <- similarity %>% 
    group_by(cell_id) %>% 
    summarize(
        cell_id=cell_id[1],
        cell_type=cell_type[1],
        mouse_id=mouse_id[1],
        greater_p_value=ifelse(
            (sum(day == 1) > 0) & (sum(day == 7) > 0),
            wilcox.test(
                tone[day == 1], 
                tone[day == 7],
                alternative='greater',
                paired=FALSE,
                na.action=na.exclude
            )$p.value,
            NA
        ),
        less_p_value=ifelse(
            (sum(day == 1) > 0) & (sum(day == 7) > 0),
            wilcox.test(
                tone[day == 1], 
                tone[day == 7],
                alternative='less',
                paired=FALSE,
                na.action=na.exclude
            )$p.value,
            NA
        )
    )

omnibus_p_vals <- tone_change_by_cell %>%
    group_by(cell_type) %>%
    summarize(
        cell_type=cell_type[1],
        num_greater=sum(greater_p_value <= 0.05, na.rm=TRUE),
        num_less=sum(less_p_value <= 0.05, na.rm=TRUE),
        num_cells=length(greater_p_value),
        greater_omnibus_p=omnibus_test(
            sum(greater_p_value <= 0.05, na.rm=TRUE),
            length(greater_p_value),
            0.05
        ),
        less_omnibus_p=omnibus_test(
            sum(less_p_value <= 0.05, na.rm=TRUE),
            length(less_p_value),
            0.05
        )
    )

omnibus_p_vals[, 'pct_greater'] <- omnibus_p_vals[, 'num_greater'] / omnibus_p_vals[, 'num_cells']
omnibus_p_vals[, 'pct_less'] <- omnibus_p_vals[, 'num_less'] / omnibus_p_vals[, 'num_cells']

omnibus_p_vals

`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)


cell_type,num_greater,num_less,num_cells,greater_omnibus_p,less_omnibus_p,pct_greater,pct_less
pv,166,8,312,1.245353e-127,9.888834e-01,0.5320513,0.02564103
pyr,178,244,1207,2.696795e-37,3.300378e-77,0.1474731,0.20215410
vip,119,26,407,1.853809e-56,1.226786e-01,0.2923833,0.06388206
